In [6]:
# !pip install pandas scikit-learn tensorflow

Defaulting to user installation because normal site-packages is not writeable
Looking in links: /usr/share/pip-wheels
  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/58/70/e8ac764ec80810eefcbab0cb1d21dbba6cf26719c44cd6d9a5e9f0407935/tensorflow-2.16.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for absl-py>=1.0.0 from https://files.pythonhosted.org/packages/a2/ad/e0d3c824784ff121c03cc031f944bc7e139a8f1870ffd2845cc2dd76f6c4/absl_py-2.1.0-py3-none-any.whl.metadata
  Obtaining dependency information for astunparse>=1.6.0 from https://files.pythonhosted.org/packages/2b/03/13dde6512ad7b4557eb792fbcf0c653af6076b81e5941d36ec61f7ce6028/astunparse-1.6.3-py2.py3-none-any.whl.metadata
  Obtaining dependency information for flatbuffers>=23.5.26 from https://files.pythonhosted.org/packages/41/f0/7e988a019bc54b2dbd0ad4182ef2d53488bb02e58694cd79d61369e85900/flatbuffers-24.3.25-py2.py3-none-any.whl

In [ ]:
# import pandas as pd
# import numpy as np
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Dropout
# from tensorflow.keras.utils import to_categorical
# # from sklearn.metrics import classification_report, confusion_matrix
# import seaborn as sns
# import matplotlib.pyplot as plt


In [ ]:
# !pip install tensorflow

In [ ]:
# # Load your dataset
# data = pd.read_csv('all_three.csv')

# # Display the first few rows of the dataset
# data.head()

# # Separate features and labels
# X = data.drop(columns=['label'])
# y = data['label']

# # Standardize the features
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X)

# # One-hot encode the labels if they are not binary
# y_encoded = to_categorical(y)

# # Split the dataset into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.2, random_state=42)


In [ ]:
# # Build the neural network model
# model = Sequential()
# model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(32, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(y_train.shape[1], activation='softmax'))

# # Compile the model
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# # Train the model
# history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))


In [ ]:
# # Evaluate the model
# score = model.evaluate(X_test, y_test)
# print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

# # Generate classification report and confusion matrix
# y_pred = model.predict(X_test)
# y_pred_classes = np.argmax(y_pred, axis=1)
# y_true = np.argmax(y_test, axis=1)

# print(classification_report(y_true, y_pred_classes))

# conf_matrix = confusion_matrix(y_true, y_pred_classes)
# sns.heatmap(conf_matrix, annot=True, fmt='d')
# plt.xlabel('Predicted')
# plt.ylabel('Actual')
# plt.show()


In [5]:
# Import necessary libraries
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense, LSTM, Concatenate
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
import networkx as nx
%pip install dgl

import dgl
from dgl.nn import GraphConv

# Load the dataset
data = pd.read_csv('all_three.csv')

# Preprocess the dataset
label_encoder = LabelEncoder()
data['label'] = label_encoder.fit_transform(data['label'])

scaler = StandardScaler()
features = ['balance', 'rec/sent', 'amount', 'size', 'weight', 'version', 
            'lock_time', 'input_count', 'output_count', 'input_total', 'output_total', 'fee', 'cdd_total']
data[features] = scaler.fit_transform(data[features])

X = data[features].values
y = data['label'].values

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Apply SMOTE to the training data
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Reshape data for LSTM (sequence length = 1 for simplicity)
X_train_seq = X_train_smote.reshape((X_train_smote.shape[0], 1, X_train_smote.shape[1]))
X_test_seq = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))
from sklearn.metrics import accuracy_score, precision_score, recall_score
from tensorflow.keras.metrics import Precision, Recall

# Functions for metrics
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    y_pred_classes = y_pred.argmax(axis=1)
    accuracy = accuracy_score(y_test, y_pred_classes)
    precision = precision_score(y_test, y_pred_classes, average='weighted')
    recall = recall_score(y_test, y_pred_classes, average='weighted')
    return accuracy, precision, recall

# ====================
# 1. Train and Test CNN
# ====================
cnn_model = Model(inputs=cnn_input, outputs=cnn_output)
cnn_model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
cnn_model.fit(X_train_seq, y_train_smote, epochs=10, batch_size=64, validation_split=0.2)
cnn_acc, cnn_prec, cnn_rec = evaluate_model(cnn_model, X_test_seq, y_test)
print(f"CNN - Accuracy: {cnn_acc}, Precision: {cnn_prec}, Recall: {cnn_rec}")

# ====================
# 2. Train and Test LSTM
# ====================
lstm_model = Model(inputs=lstm_input, outputs=lstm_output)
lstm_model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
lstm_model.fit(X_train_seq, y_train_smote, epochs=10, batch_size=64, validation_split=0.2)
lstm_acc, lstm_prec, lstm_rec = evaluate_model(lstm_model, X_test_seq, y_test)
print(f"LSTM - Accuracy: {lstm_acc}, Precision: {lstm_prec}, Recall: {lstm_rec}")

# ====================
# 3. Train and Test GNN
# ====================
class GNNModel(tf.keras.Model):
    def __init__(self, in_feats, h_feats):
        super(GNNModel, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, h_feats)

    def call(self, g, inputs):
        h = self.conv1(g, inputs)
        h = tf.nn.relu(h)
        h = self.conv2(g, h)
        return h

gnn_features_train = tf.convert_to_tensor(X_train_smote, dtype=tf.float32)
gnn_features_test = tf.convert_to_tensor(X_test, dtype=tf.float32)
gnn_model = GNNModel(in_feats=gnn_features_train.shape[1], h_feats=64)
gnn_output_train = gnn_model(dgl_graph, gnn_features_train)
gnn_output_test = gnn_model(dgl_graph, gnn_features_test)

# Train a dense classifier on GNN outputs
gnn_classifier = Dense(len(label_encoder.classes_), activation='softmax')
gnn_model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
gnn_model.fit(gnn_output_train, y_train_smote, epochs=10, batch_size=64, validation_split=0.2)
gnn_acc, gnn_prec, gnn_rec = evaluate_model(gnn_model, gnn_output_test, y_test)
print(f"GNN - Accuracy: {gnn_acc}, Precision: {gnn_prec}, Recall: {gnn_rec}")

# ====================
# 4. Train and Test Hybrid Model
# ====================
combined_output = Concatenate()([cnn_output, lstm_output, gnn_output])
final_output = Dense(len(label_encoder.classes_), activation='softmax')(combined_output)
hybrid_model = Model(inputs=[cnn_input, lstm_input], outputs=final_output)

hybrid_model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history = hybrid_model.fit([X_train_seq, X_train_seq], y_train_smote, epochs=10, batch_size=64, validation_split=0.2)
hybrid_acc, hybrid_prec, hybrid_rec = evaluate_model(hybrid_model, [X_test_seq, X_test_seq], y_test)
print(f"Hybrid Model - Accuracy: {hybrid_acc}, Precision: {hybrid_prec}, Recall: {hybrid_rec}")


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


FileNotFoundError: Could not find module 'c:\Users\admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\dgl\dgl.dll' (or one of its dependencies). Try using the full path with constructor syntax.

In [2]:
# !pip install imblearn
!pip uninstall -y dgl dgl-cu* dgl-cuda* dgl-cpu

ERROR: Invalid requirement: 'dgl-cu*'


In [3]:
# Uninstall any existing DGL installations
!pip uninstall -y dgl dgl-cuda dgl-cpu
 
# Install the correct DGL version for Windows and PyTorch
!pip install dgl -f https://data.dgl.ai/wheels/repo.html
 
# After running this cell, restart the kernel before importing dgl

Found existing installation: dgl 2.2.1
Uninstalling dgl-2.2.1:
  Successfully uninstalled dgl-2.2.1


Looking in links: https://data.dgl.ai/wheels/repo.html
     ---------------------------------------- 0.0/82.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/82.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/82.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/82.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/82.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/82.3 MB ? eta -:--:--
     --------------------------------------- 0.1/82.3 MB 233.8 kB/s eta 0:05:52
     --------------------------------------- 0.1/82.3 MB 233.8 kB/s eta 0:05:52
     --------------------------------------- 0.1/82.3 MB 233.8 kB/s eta 0:05:52
     --------------------------------------- 0.1/82.3 MB 233.8 kB/s eta 0:05:52
     --------------------------------------- 0.1/82.3 MB 233.8 kB/s eta 0:05:52
     --------------------------------------- 0.1/82.3 MB 293.9 kB/s eta 0:04:40
     ----------------------------


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import dgl
from dgl.nn import GraphConv

FileNotFoundError: Could not find module 'c:\Users\admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\dgl\dgl.dll' (or one of its dependencies). Try using the full path with constructor syntax.

In [7]:
# Uninstall DGL if present
!pip uninstall -y dgl dgl-cu* dgl-cuda* dgl-cpu
 
# Install PyTorch Geometric and dependencies for Windows (CPU version)
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
!pip install torch-scatter -f https://data.pyg.org/whl/torch-2.1.0+cpu.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-2.1.0+cpu.html
!pip install torch-geometric

ERROR: Invalid requirement: 'dgl-cu*'


Looking in indexes: https://download.pytorch.org/whl/cpu
INFO: pip is looking at multiple versions of torchaudio to determine which version is compatible with other requirements. This could take a while.
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ------------ --------------------------- 0.7/2.5 MB 23.0 MB/s eta 0:00:01
   --------------- ------------------------ 0.9/2.5 MB 29.1 MB/s eta 0:00:01
   ------------------------------------- -- 2.4/2.5 MB 18.8 MB/s eta 0:00:01
   ---------------------------------------- 2.5/2.5 MB 17.5 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Looking in links: https://data.pyg.org/whl/torch-2.1.0+cpu.html
     ---------------------------------------- 0.0/108.0 kB ? eta -:--:--
     --- ------------------------------------ 10.2/108.0 kB ? eta -:--:--
     -------------- ---------------------- 41.0/108.0 kB 326.8 kB/s eta 0:00:01
     ------------------------ ------------ 71.7/108.0 kB 491.5 kB/s eta 0:00:01
     ------------------------------------ 108.0/108.0 kB 693.7 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for torch-scatter
Failed to build torch-scatter


  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [44 lines of output]
      C:\Users\admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\setuptools\dist.py:759: SetuptoolsDeprecationWarning: License classifiers are deprecated.
      !!
      
              ********************************************************************************
              Please consider removing the following classifiers in favor of a SPDX license expression:
      
              License :: OSI Approved :: MIT License
      
              See https://packaging.python.org/en/latest/guides/writing-pyproject-toml/#license for details.
              ********************************************************************************
      
      !!
        self._finalize_license_expression()
      running bdist_wheel
      running build
      running build_py
      creating build\lib.win-amd64-cpython-312\torch_scatter
      copyin

Looking in links: https://data.pyg.org/whl/torch-2.1.0+cpu.html
     ---------------------------------------- 0.0/210.0 kB ? eta -:--:--
     ----- ------------------------------- 30.7/210.0 kB 660.6 kB/s eta 0:00:01
     ----------------- --------------------- 92.2/210.0 kB 1.1 MB/s eta 0:00:01
     -------------------------------------- 210.0/210.0 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for torch-sparse
Failed to build torch-sparse


  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [69 lines of output]
      C:\Users\admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\setuptools\dist.py:759: SetuptoolsDeprecationWarning: License classifiers are deprecated.
      !!
      
              ********************************************************************************
              Please consider removing the following classifiers in favor of a SPDX license expression:
      
              License :: OSI Approved :: MIT License
      
              See https://packaging.python.org/en/latest/guides/writing-pyproject-toml/#license for details.
              ********************************************************************************
      
      !!
        self._finalize_license_expression()
      running bdist_wheel
      C:\Users\admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\cpp_extension.py:576: Use

     ---------------------------------------- 0.0/63.1 kB ? eta -:--:--
     ------------ ------------------------- 20.5/63.1 kB 330.3 kB/s eta 0:00:01
     ------------------------------ ------- 51.2/63.1 kB 525.1 kB/s eta 0:00:01
     -------------------------------------- 63.1/63.1 kB 561.9 kB/s eta 0:00:00
   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   -- ------------------------------------- 0.1/1.1 MB 3.2 MB/s eta 0:00:01
   ----- ---------------------------------- 0.1/1.1 MB 2.8 MB/s eta 0:00:01
   -------- ------------------------------- 0.2/1.1 MB 2.4 MB/s eta 0:00:01
   ------------ --------------------------- 0.4/1.1 MB 2.5 MB/s eta 0:00:01
   ---------------- ----------------------- 0.5/1.1 MB 2.6 MB/s eta 0:00:01
   ------------------- -------------------- 0.6/1.1 MB 2.5 MB/s eta 0:00:01
   ----------------------- ---------------- 0.7/1.1 MB 2.6 MB/s eta 0:00:01
   -------------------------- ------------- 0.8/1.1 MB 2.7 MB/s eta 0:00:01
   --------


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip
